# 1. Formula 1 World Championship (1950 - 2024) Analysis 

<img src="f1_ferrari3.png" style="width:60%; border-radius:20px;" />

### 1.1 Environment Setup

##### Import of the packages

In [1]:
%pip install --upgrade google-cloud
%pip install --upgrade google-cloud-monitoring
%pip install --upgrade google-cloud-storage
%pip install --upgrade google-cloud-bigquery
%pip install --upgrade google-cloud-bigquery-storage
%pip install --upgrade pyspark
import pyspark
import findspark
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from google.cloud import bigquery
from pyspark.sql import SparkSession


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


##### Displaying the path of Google Cloud credentials file for authentication

In [2]:
!echo %GOOGLE_APPLICATION_CREDENTIALS%

%GOOGLE_APPLICATION_CREDENTIALS%


##### Setting Google Cloud Credentials and Initializing BigQuery Client

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\Alessandro\Downloads\ccbd-exam-2025-risino-1e4ecafc3ece.json"

!echo %GOOGLE_APPLICATION_CREDENTIALS%
client = bigquery.Client() 
print('done')

C:\Users\Alessandro\Downloads\ccbd-exam-2025-risino-1e4ecafc3ece.json
done


In [4]:
!findstr "project_id" "%GOOGLE_APPLICATION_CREDENTIALS%"

  "project_id": "ccbd-exam-2025-risino",


### 1.2 Dataset Exploration and Selection methods

##### Selection of the Datasets from GCP with the first method

Fundamental the "client = bigquery.Client()" for this method

In [5]:
circuits = "SELECT * FROM `formula1.circuits` LIMIT 5"
client.query(circuits).to_dataframe()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,25,galvez,Autódromo Juan y Oscar Gálvez,Buenos Aires,Argentina,-34.6943,-58.4593,8,http://en.wikipedia.org/wiki/Aut%C3%B3dromo_Os...
1,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
2,29,adelaide,Adelaide Street Circuit,Adelaide,Australia,-34.9272,138.6170,58,http://en.wikipedia.org/wiki/Adelaide_Street_C...
3,57,zeltweg,Zeltweg,Styria,Austria,47.2039,14.7478,676,http://en.wikipedia.org/wiki/Zeltweg_Airfield
4,70,red_bull_ring,Red Bull Ring,Spielberg,Austria,47.2197,14.7647,678,http://en.wikipedia.org/wiki/Red_Bull_Ring


In [6]:
constructors = "SELECT * FROM `formula1.constructors` LIMIT 5"
client.query(constructors).to_dataframe()

,constructorId,constructorRef,name,nationality,url
0,67,penske,Penske,American,http://en.wikipedia.org/wiki/Penske_Racing
1,73,parnelli,Parnelli,American,http://en.wikipedia.org/wiki/Parnelli
2,88,eagle,Eagle,American,http://en.wikipedia.org/wiki/Anglo_American_Ra...
3,106,scarab,Scarab,American,http://en.wikipedia.org/wiki/Scarab_(constructor)
4,107,watson,Watson,American,http://en.wikipedia.org/wiki/A.J._Watson


In [7]:
drivers = "SELECT * FROM `formula1.drivers` LIMIT 5"
client.query(drivers).to_dataframe()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,31,montoya,\N,MON,Juan,Pablo Montoya,1975-09-20,Colombian,http://en.wikipedia.org/wiki/Juan_Pablo_Montoya
1,39,karthikeyan,\N,KAR,Narain,Karthikeyan,1977-01-14,Indian,http://en.wikipedia.org/wiki/Narain_Karthikeyan
2,47,baumgartner,\N,\N,Zsolt,Baumgartner,1981-01-01,Hungarian,http://en.wikipedia.org/wiki/Zsolt_Baumgartner
3,61,enge,\N,\N,Tomáš,Enge,1976-09-11,Czech,http://en.wikipedia.org/wiki/Tom%C3%A1%C5%A1_Enge
4,194,salazar,\N,\N,Eliseo,Salazar,1954-11-14,Chilean,http://en.wikipedia.org/wiki/Eliseo_Salazar


In [8]:
races = "SELECT * FROM `formula1.races` LIMIT 5"
client.query(races).to_dataframe()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1100,2023,3,1,Australian Grand Prix,2023-04-02,05:00:00,https://en.wikipedia.org/wiki/2023_Australian_...,2023-03-31,01:30:00,2023-03-31,05:00:00,2023-04-01,01:30:00,2023-04-01,05:00:00,\N,\N
1,1123,2024,3,1,Australian Grand Prix,2024-03-24,04:00:00,https://en.wikipedia.org/wiki/2024_Australian_...,2024-03-22,01:30:00,2024-03-22,05:00:00,2024-03-23,01:30:00,2024-03-23,05:00:00,\N,\N
2,1076,2022,3,1,Australian Grand Prix,2022-04-10,05:00:00,http://en.wikipedia.org/wiki/2022_Australian_G...,2022-04-08,03:00:00,2022-04-08,06:00:00,2022-04-09,03:00:00,2022-04-09,06:00:00,\N,\N
3,1092,2022,18,22,Japanese Grand Prix,2022-10-09,05:00:00,http://en.wikipedia.org/wiki/2022_Japanese_Gra...,2022-10-07,03:00:00,2022-10-07,06:00:00,2022-10-08,03:00:00,2022-10-08,06:00:00,\N,\N
4,1114,2023,16,22,Japanese Grand Prix,2023-09-24,05:00:00,https://en.wikipedia.org/wiki/2023_Japanese_Gr...,2023-09-22,02:30:00,2023-09-22,06:00:00,2023-09-23,02:30:00,2023-09-23,06:00:00,\N,\N


In [9]:
results = "SELECT * FROM `formula1.results` LIMIT 5"
client.query(results).to_dataframe()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,5249,261,71,3,0,2,1,1,1,10.0,65,1:36:14.3,5774300,\N,\N,\N,\N,1
1,5331,264,71,3,0,1,1,1,1,10.0,60,1:30:03.640,5403640,\N,\N,\N,\N,1
2,5415,267,71,3,0,3,1,1,1,10.0,44,1:28:47.1,5327100,\N,\N,\N,\N,1
3,5443,268,71,3,0,3,1,1,1,10.0,53,1:18:03.2,4683200,\N,\N,\N,\N,1
4,5471,269,71,3,0,2,1,1,1,10.0,71,1:45:10.1,6310100,\N,\N,\N,\N,1


##### Import and test of BigQuery Magics (faster)

In [10]:
%pip install bigquery-magics
%load_ext bigquery_magics
!ipython locate
# %reload_ext bigquery_magics "re-load" the extension bigquery_magics if we have modified something o we want to see that it is active

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\Alessandro\.ipython


In [11]:
%%bigquery
SELECT * FROM `formula1.results` LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,5249,261,71,3,0,2,1,1,1,10.0,65,1:36:14.3,5774300,\N,\N,\N,\N,1
1,5331,264,71,3,0,1,1,1,1,10.0,60,1:30:03.640,5403640,\N,\N,\N,\N,1
2,5415,267,71,3,0,3,1,1,1,10.0,44,1:28:47.1,5327100,\N,\N,\N,\N,1
3,5443,268,71,3,0,3,1,1,1,10.0,53,1:18:03.2,4683200,\N,\N,\N,\N,1
4,5471,269,71,3,0,2,1,1,1,10.0,71,1:45:10.1,6310100,\N,\N,\N,\N,1


### 1.3 Join of the Datasets: creation of "joined_results"

##### Join of the 5 chosen Datasets:
"Circuits", "Constructors", "Drivers", "Races" and "Results"

In [12]:
%%bigquery
CREATE OR REPLACE TABLE `formula1.joined_results` AS 
SELECT
  c.circuitId AS circuit_id,
  c.name AS circuit_name,
  c.location AS circuit_location,
  c.country AS circuit_country,
  c.lat AS circuit_latitude,
  c.lng AS circuit_longitude,
  c.alt AS circuit_altitude,

  r.raceId AS race_id,
  r.name AS race_name,
  r.year AS race_year,
  r.date AS race_date,

  d.driverId AS driver_id,
  CONCAT(d.forename, ' ', d.surname) AS driver_name,
  d.dob AS driver_dob,
  d.nationality AS driver_nationality,

  cons.constructorId AS constructor_id,
  cons.name AS constructor_name,
  cons.nationality AS constructor_nationality,

  res.resultId AS result_id,
  res.number AS car_number,
  res.grid AS start_grid_position,
  res.position AS final_position,
  res.points AS race_points,
  res.laps AS total_laps,
  res.fastestLap AS fastest_lap_number,
  res.fastestLapTime AS fastest_lap_time,
  res.fastestLapSpeed AS fastest_lap_speed

FROM `formula1.results` res
JOIN `formula1.races` r
  ON res.raceId = r.raceId
JOIN `formula1.circuits` c
  ON r.circuitId = c.circuitId
JOIN `formula1.drivers` d
  ON res.driverId = d.driverId
JOIN `formula1.constructors` cons
  ON res.constructorId = cons.constructorId

Query is running:   0%|          |

""


##### Pandas Loading of the joined dataset

In [13]:
%%bigquery df_f1
SELECT * FROM `formula1.joined_results`

Query is running:   0%|          |

Downloading:   0%|          |

In [14]:
display(df_f1)

,circuit_id,circuit_name,circuit_location,circuit_country,circuit_latitude,circuit_longitude,circuit_altitude,race_id,race_name,race_year,...,constructor_nationality,result_id,car_number,start_grid_position,final_position,race_points,total_laps,fastest_lap_number,fastest_lap_time,fastest_lap_speed
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,French,7570,8,14,\N,0.0,24,17,1:30.502,210.943
1,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,Japanese,7557,10,19,4,5.0,58,53,1:28.416,215.920
2,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,British,7559,16,5,6,3.0,58,48,1:27.706,217.668
3,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,Italian,7561,11,17,8,1.0,58,50,1:29.823,212.537
4,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,British,7572,2,12,\N,0.0,0,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26748,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,British,26723,23,17,\N,0.0,25,21,1:38.008,227.773
26749,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,British,26710,4,6,6,9.0,50,50,1:34.876,235.292
26750,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,French,26721,31,11,17,0.0,49,47,1:36.511,231.306
26751,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,Swiss,26722,77,19,18,0.0,49,46,1:36.601,231.090


##### Cleaning the joined dataset

In [15]:
df_f1.dtypes

circuit_id                   Int64
circuit_name                object
circuit_location            object
circuit_country             object
circuit_latitude           float64
circuit_longitude          float64
circuit_altitude             Int64
race_id                      Int64
race_name                   object
race_year                    Int64
race_date                   dbdate
driver_id                    Int64
driver_name                 object
driver_dob                  dbdate
driver_nationality          object
constructor_id               Int64
constructor_name            object
constructor_nationality     object
result_id                    Int64
car_number                   Int64
start_grid_position          Int64
final_position              object
race_points                float64
total_laps                   Int64
fastest_lap_number          object
fastest_lap_time            object
fastest_lap_speed           object
dtype: object

In [16]:
# dtypes conversion
df_f1['final_position'] = pd.to_numeric(df_f1['final_position'], errors='coerce')
df_f1['fastest_lap_number'] = pd.to_numeric(df_f1['fastest_lap_number'], errors='coerce')
df_f1['fastest_lap_speed'] = (
    df_f1['fastest_lap_speed']
    .str.replace(',', '.', regex=False)  # sostituisci virgole con punti
)
df_f1['fastest_lap_speed'] = pd.to_numeric(df_f1['fastest_lap_speed'], errors='coerce')
# coerce converte ignorando gli errori ('\\N' diventa NaN)

In [17]:
df_f1.dtypes

circuit_id                   Int64
circuit_name                object
circuit_location            object
circuit_country             object
circuit_latitude           float64
circuit_longitude          float64
circuit_altitude             Int64
race_id                      Int64
race_name                   object
race_year                    Int64
race_date                   dbdate
driver_id                    Int64
driver_name                 object
driver_dob                  dbdate
driver_nationality          object
constructor_id               Int64
constructor_name            object
constructor_nationality     object
result_id                    Int64
car_number                   Int64
start_grid_position          Int64
final_position             float64
race_points                float64
total_laps                   Int64
fastest_lap_number         float64
fastest_lap_time            object
fastest_lap_speed          float64
dtype: object

In [18]:
df_f1.describe()

,circuit_id,circuit_latitude,circuit_longitude,circuit_altitude,race_id,race_year,driver_id,constructor_id,result_id,car_number,start_grid_position,final_position,race_points,total_laps,fastest_lap_number,fastest_lap_speed
count,26753.0,26753.000000,26753.000000,26753.0,26753.0,26753.0,26753.0,26753.0,26753.0,26753.0,26753.0,15806.000000,26753.000000,26753.0,8252.000000,8252.000000
mean,23.814077,33.949189,5.724360,276.893545,551.646656,1991.400964,278.639069,50.165589,13379.899151,18.153927,11.137293,8.020499,1.988078,46.312152,42.732913,204.116330
std,19.10885,25.244714,57.639049,409.12065,313.28841,19.950263,282.723999,61.544163,7726.652173,15.581135,7.201737,4.840796,4.351595,29.491712,16.603460,21.377265
min,1.0,-37.849700,-118.189000,-7.0,1.0,1950.0,1.0,1.0,1.0,0.0,0.0,1.000000,0.000000,0.0,1.000000,89.540000
25%,9.0,34.843100,-1.016940,18.0,300.0,1977.0,57.0,6.0,6689.0,7.0,5.0,4.000000,0.000000,23.0,33.000000,193.322500
50%,18.0,43.734700,6.947500,153.0,531.0,1991.0,172.0,25.0,13377.0,16.0,11.0,8.000000,0.000000,53.0,46.000000,204.894000
75%,34.0,49.254200,14.764700,401.0,811.0,2009.0,399.0,63.0,20070.0,24.0,17.0,11.000000,2.000000,66.0,54.000000,217.324000
max,80.0,57.265300,144.968000,2227.0,1144.0,2024.0,862.0,215.0,26764.0,208.0,34.0,33.000000,50.000000,200.0,85.000000,257.320000


In [19]:
from google.cloud import bigquery
client = bigquery.Client()
table_id = "formula1.joined_results"
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
job = client.load_table_from_dataframe(df_f1, table_id, job_config=job_config) # Crea una configurazione di caricamento (LoadJobConfig) dove:
# write_disposition="WRITE_TRUNCATE" dice a BigQuery:
# sovrascrive completamente la tabella esistente (TRUNCATE = tronca).
job.result()  

LoadJob<project=ccbd-exam-2025-risino, location=EU, id=189c9420-2464-4f5f-9ca2-499602d1cd34>

In [20]:
display(df_f1)

,circuit_id,circuit_name,circuit_location,circuit_country,circuit_latitude,circuit_longitude,circuit_altitude,race_id,race_name,race_year,...,constructor_nationality,result_id,car_number,start_grid_position,final_position,race_points,total_laps,fastest_lap_number,fastest_lap_time,fastest_lap_speed
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,French,7570,8,14,NaN,0.0,24,17.0,1:30.502,210.943
1,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,Japanese,7557,10,19,4.0,5.0,58,53.0,1:28.416,215.920
2,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,British,7559,16,5,6.0,3.0,58,48.0,1:27.706,217.668
3,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,Italian,7561,11,17,8.0,1.0,58,50.0,1:29.823,212.537
4,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,British,7572,2,12,NaN,0.0,0,NaN,\N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26748,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,British,26723,23,17,NaN,0.0,25,21.0,1:38.008,227.773
26749,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,British,26710,4,6,6.0,9.0,50,50.0,1:34.876,235.292
26750,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,French,26721,31,11,17.0,0.0,49,47.0,1:36.511,231.306
26751,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,Swiss,26722,77,19,18.0,0.0,49,46.0,1:36.601,231.090


### 1.4 Basic Insights

##### Distinct Counts

In [21]:
%%bigquery
SELECT COUNT(DISTINCT race_id) AS distinct_race_ids FROM `formula1.joined_results`

Query is running:   0%|          |

Downloading:   0%|          |

,distinct_race_ids
0,1125


In [22]:
%%bigquery
SELECT COUNT(DISTINCT constructor_id) AS distinct_constructor_ids FROM `formula1.joined_results`

Query is running:   0%|          |

Downloading:   0%|          |

,distinct_constructor_ids
0,211


In [23]:
%%bigquery
SELECT COUNT(DISTINCT driver_id) AS distinct_driver_ids FROM `formula1.joined_results`

Query is running:   0%|          |

Downloading:   0%|          |

,distinct_driver_ids
0,860


In [24]:
%%bigquery
SELECT COUNT(DISTINCT circuit_id) AS distinct_circuit_ids FROM `formula1.joined_results`

Query is running:   0%|          |

Downloading:   0%|          |

,distinct_circuit_ids
0,77


In [25]:
%%bigquery
SELECT COUNT(final_position) AS count_first_positions FROM `formula1.joined_results`WHERE final_position = 1

Query is running:   0%|          |

Downloading:   0%|          |

,count_first_positions
0,1128


##### Basic Queries

In [26]:
%%bigquery
SELECT constructor_name,
  COUNT(*) AS total_constructor_wins
FROM
  formula1.joined_results
WHERE
  wins = 1
GROUP BY
  constructor_name
ORDER BY
  total_constructor_wins DESC;

Executing query with job ID: bc2c102d-e542-468a-ad2b-8374048ea1df
Query executing: 0.32s


ERROR:
 400 Unrecognized name: wins at [6:3]; reason: invalidQuery, location: query, message: Unrecognized name: wins at [6:3]

Location: EU
Job ID: bc2c102d-e542-468a-ad2b-8374048ea1df



In [27]:
%%bigquery
SELECT
  driver_name,
  driver_nationality,
  COUNT(*) AS total_wins
FROM
  formula1.joined_results
WHERE
  wins = 1
GROUP BY
  driver_name, driver_nationality
ORDER BY
  total_wins DESC;

Executing query with job ID: ed07ef18-f4b2-42fc-87e9-5ae4e9305ce6
Query executing: 0.45s


ERROR:
 400 Unrecognized name: wins at [8:3]; reason: invalidQuery, location: query, message: Unrecognized name: wins at [8:3]

Location: EU
Job ID: ed07ef18-f4b2-42fc-87e9-5ae4e9305ce6



In [28]:
%%bigquery
SELECT
  circuit_name,
  circuit_country,
  MAX(fastest_lap_speed) AS max_fastest_lap_speed
FROM
  formula1.joined_results
WHERE
  fastest_lap_speed IS NOT NULL
GROUP BY
  circuit_name, circuit_country
ORDER BY
  max_fastest_lap_speed DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,circuit_name,circuit_country,max_fastest_lap_speed
0,Autodromo Nazionale di Monza,Italy,257.320
1,Jeddah Corniche Circuit,Saudi Arabia,244.962
2,Silverstone Circuit,UK,243.494
3,Circuit de Spa-Francorchamps,Belgium,240.822
4,Autodromo Internazionale del Mugello,Italy,239.518
5,Albert Park Grand Prix Circuit,Australia,238.066
6,Red Bull Ring,Austria,236.894
7,Losail International Circuit,Qatar,236.798
8,Las Vegas Strip Street Circuit,United States,235.292
9,Autodromo Enzo e Dino Ferrari,Italy,234.121


### 1.5 Adding "wins" column

In [29]:
%%bigquery df_f1
SELECT *, IF(final_position = 1, 1, 0) AS wins
FROM formula1.joined_results

Query is running:   0%|          |

Downloading:   0%|          |

In [30]:
from google.cloud import bigquery
client = bigquery.Client()
table_id = "formula1.joined_results"
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
job = client.load_table_from_dataframe(df_f1, table_id, job_config=job_config) # Create a load configuration (LoadJobConfig) where:
# write_disposition="WRITE_TRUNCATE" tells BigQuery:
# completely overwrite the existing table (TRUNCATE = cut off).
job.result()  

LoadJob<project=ccbd-exam-2025-risino, location=EU, id=04663e22-28f6-4534-a94e-d0ab4a58217b>

In [31]:
display(df_f1)

,circuit_id,circuit_name,circuit_location,circuit_country,circuit_latitude,circuit_longitude,circuit_altitude,race_id,race_name,race_year,...,result_id,car_number,start_grid_position,final_position,race_points,total_laps,fastest_lap_number,fastest_lap_time,fastest_lap_speed,wins
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,7570,8,14,NaN,0.0,24,17.0,1:30.502,210.943,0
1,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,7557,10,19,4.0,5.0,58,53.0,1:28.416,215.920,0
2,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,7559,16,5,6.0,3.0,58,48.0,1:27.706,217.668,0
3,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,7561,11,17,8.0,1.0,58,50.0,1:29.823,212.537,0
4,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1,Australian Grand Prix,2009,...,7572,2,12,NaN,0.0,0,NaN,\N,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26748,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,26723,23,17,NaN,0.0,25,21.0,1:38.008,227.773,0
26749,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,26710,4,6,6.0,9.0,50,50.0,1:34.876,235.292,0
26750,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,26721,31,11,17.0,0.0,49,47.0,1:36.511,231.306,0
26751,80,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.173,642,1142,Las Vegas Grand Prix,2024,...,26722,77,19,18.0,0.0,49,46.0,1:36.601,231.090,0


In [32]:
df_f1.describe()

,circuit_id,circuit_latitude,circuit_longitude,circuit_altitude,race_id,race_year,driver_id,constructor_id,result_id,car_number,start_grid_position,final_position,race_points,total_laps,fastest_lap_number,fastest_lap_speed,wins
count,26753.0,26753.000000,26753.000000,26753.0,26753.0,26753.0,26753.0,26753.0,26753.0,26753.0,26753.0,15806.000000,26753.000000,26753.0,8252.000000,8252.000000,26753.0
mean,23.814077,33.949189,5.724360,276.893545,551.646656,1991.400964,278.639069,50.165589,13379.899151,18.153927,11.137293,8.020499,1.988078,46.312152,42.732913,204.116330,0.042163
std,19.10885,25.244714,57.639049,409.12065,313.28841,19.950263,282.723999,61.544163,7726.652173,15.581135,7.201737,4.840796,4.351595,29.491712,16.603460,21.377265,0.200966
min,1.0,-37.849700,-118.189000,-7.0,1.0,1950.0,1.0,1.0,1.0,0.0,0.0,1.000000,0.000000,0.0,1.000000,89.540000,0.0
25%,9.0,34.843100,-1.016940,18.0,300.0,1977.0,57.0,6.0,6689.0,7.0,5.0,4.000000,0.000000,23.0,33.000000,193.322500,0.0
50%,18.0,43.734700,6.947500,153.0,531.0,1991.0,172.0,25.0,13377.0,16.0,11.0,8.000000,0.000000,53.0,46.000000,204.894000,0.0
75%,34.0,49.254200,14.764700,401.0,811.0,2009.0,399.0,63.0,20070.0,24.0,17.0,11.000000,2.000000,66.0,54.000000,217.324000,0.0
max,80.0,57.265300,144.968000,2227.0,1144.0,2024.0,862.0,215.0,26764.0,208.0,34.0,33.000000,50.000000,200.0,85.000000,257.320000,1.0


### 1.6 Discrepancy between Races and Wins (1st interesting insight)

In [33]:
%%bigquery
SELECT
  (SELECT COUNT(DISTINCT race_id) FROM `formula1.joined_results`) AS num_races,
  (SELECT COUNT(*) FROM `formula1.joined_results` WHERE wins = 1) AS wins_equal_1

Query is running:   0%|          |

Downloading:   0%|          |

,num_races,wins_equal_1
0,1125,1128


**There is a slight discrepancy between the data (the number of wins should theoretically match the number of races), but we could look for a reason in this discrepancy.**

In [34]:
%%bigquery
SELECT
  race_id,
  race_year,
  race_name,
  COUNT(*) AS win_count,
  STRING_AGG(driver_name, ', ') AS winners
FROM `formula1.joined_results`
WHERE final_position = 1
GROUP BY race_id, race_year, race_name
HAVING COUNT(*) > 1
ORDER BY race_year

Query is running:   0%|          |

Downloading:   0%|          |

,race_id,race_year,race_name,win_count,winners
0,828,1951,French Grand Prix,2,"Juan Fangio, Luigi Fagioli"
1,784,1956,Argentine Grand Prix,2,"Luigi Musso, Juan Fangio"
2,780,1957,British Grand Prix,2,"Stirling Moss, Tony Brooks"


##### And here's the reason: the discrepancy is due to a few historical cases in which shared drives were allowed. 
In the early years of Formula 1, it was possible for two drivers to share the same car during a race and both be credited with the win. This occurred, for example, in the the 1951 French Grand Prix, the 1956 Argentine Grand Prix and the 1957 British Grand Prix.

Sources: [F1 France 1951](https://en.wikipedia.org/wiki/1951_French_Grand_Prix), [F1 Argentina 1956](https://en.wikipedia.org/wiki/1956_Argentine_Grand_Prix), [F1 Britain 1957](https://en.wikipedia.org/wiki/1957_British_Grand_Prix)

### 1.7 Number of drivers per race and creation of "constructor_unusual_drivers"

In [35]:
%%bigquery
SELECT
  constructor_name,
  COUNT(DISTINCT race_id) AS num_races,
  COUNT(driver_id) AS total_drivers_participation
FROM
  `formula1.joined_results`
WHERE
  constructor_name IN ('Ferrari', 'McLaren')
GROUP BY
  constructor_name;

Query is running:   0%|          |

Downloading:   0%|          |

,constructor_name,num_races,total_drivers_participation
0,McLaren,929,1923
1,Ferrari,1100,2439


In [36]:
%%bigquery
WITH piloti_per_gara AS (
  SELECT
    constructor_name,
    race_id,
    COUNT(DISTINCT driver_id) AS num_drivers
  FROM
    `formula1.joined_results`
  WHERE
    constructor_name IN ('Ferrari', 'McLaren')
  GROUP BY
    constructor_name,
    race_id
)

SELECT
  constructor_name,
  num_drivers,
  COUNT(*) AS count_races
FROM
  piloti_per_gara
GROUP BY
  constructor_name,
  num_drivers
ORDER BY
  constructor_name,
  num_drivers

Query is running:   0%|          |

Downloading:   0%|          |

,constructor_name,num_drivers,count_races
0,Ferrari,1,39
1,Ferrari,2,948
2,Ferrari,3,41
3,Ferrari,4,34
4,Ferrari,5,20
5,Ferrari,6,10
6,Ferrari,7,4
7,Ferrari,8,2
8,Ferrari,9,1
9,Ferrari,10,1


In [37]:
%%bigquery
SELECT
  COUNT(*) AS total_races_not_2_drivers
FROM (
  SELECT
    race_year,
    race_name,
    constructor_name,
    COUNT(DISTINCT driver_id) AS num_drivers
  FROM
    `formula1.joined_results`
  GROUP BY
    race_year,
    race_name,
    constructor_name
  HAVING
    num_drivers != 2
)

Query is running:   0%|          |

Downloading:   0%|          |

,total_races_not_2_drivers
0,2598


In [38]:
%%bigquery
SELECT
  constructor_name,
  COUNT(*) AS num_races_not_2_drivers
FROM (
  SELECT
    race_year,
    race_name,
    constructor_name,
    COUNT(DISTINCT driver_id) AS num_drivers
  FROM
    `formula1.joined_results`
  GROUP BY
    race_year,
    race_name,
    constructor_name
  HAVING
    num_drivers != 2
)
GROUP BY
  constructor_name
ORDER BY
  num_races_not_2_drivers DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,constructor_name,num_races_not_2_drivers
0,BRM,153
1,Ferrari,152
2,Ensign,112
3,March,106
4,Osella,92
...,...,...
168,Cooper-OSCA,1
169,Tec-Mec,1
170,Cooper-Borgward,1
171,Connew,1


In [39]:
%%bigquery
CREATE OR REPLACE TABLE `formula1.constructor_unusual_drivers` AS
SELECT
  race_year,
  race_name,
  constructor_name,
  COUNT(DISTINCT driver_id) AS num_drivers,
  STRING_AGG(DISTINCT driver_name, ', ') AS drivers_list
FROM
  `formula1.joined_results`
GROUP BY
  race_year,
  race_name,
  constructor_name
HAVING
  num_drivers != 2

Query is running:   0%|          |

""


### 1.8 Details on races with number of drivers !=2 (2nd interesting insight)

In [40]:
%%bigquery
WITH unusual_drivers AS (
  SELECT *
  FROM `formula1.constructor_unusual_drivers`
  WHERE num_drivers != 2
)

SELECT
  COUNTIF(race_year < 1985) AS count_before_1985,
  COUNT(*) AS total_races_not_2_drivers,
  ROUND( (COUNTIF(race_year < 1985) / COUNT(*)) * 100, 2) AS percentage_before_1985
FROM unusual_drivers

Query is running:   0%|          |

Downloading:   0%|          |

,count_before_1985,total_races_not_2_drivers,percentage_before_1985
0,2281,2598,87.8


#### Even though 1985 is in the middle between 1950 and 2024 we can see that more than 7/8 of the cases were all before 1985. 
In fact the number of cars entered by one team into a single race was not strictly limited in the 1950s and early 1960s. Since the 1963 season teams were generally allowed to enter only two regular cars, with the third car reserved for an occasional driver. This rule was further promoted in the 1974 season when the permanent racing numbers were assigned to each team in pairs, with the third car having the racing number out of the pair. Entering more than three cars was exceptionally tolerated, most notably regarding the BRM team in the 1971 and 1972 seasons. However, many teams during this period entered only two cars, e.g. Ferrari have entered no more than two cars (with one exception at the 1976 Italian Grand Prix in connection with Lauda's comeback) every season since 1973. Since the 1985 season the FIA have required that teams enter no more than two cars for a race; during this season Renault became the last team to have entered three cars for a race at the 1985 German Grand Prix, but only two of their cars were eligible for championship points.

Source: [List of Formula One constructors (paragraph "Teams")](https://en.wikipedia.org/wiki/List_of_Formula_One_constructors#:~:text=Since%20the%201985%20season%20the,were%20eligible%20for%20championship%20points.)

#### In our dataset we are lucky to see exactly this exception for Renault

In [41]:
%%bigquery 
SELECT * FROM formula1.constructor_unusual_drivers
WHERE race_year >= 1985 
AND num_drivers >2

Query is running:   0%|          |

Downloading:   0%|          |

,race_year,race_name,constructor_name,num_drivers,drivers_list
0,1985,German Grand Prix,Renault,3,"Patrick Tambay, Derek Warwick, François Hesnault"


In fact

In [42]:
%%bigquery
WITH post_1985_unusual_drivers AS (
  SELECT *
  FROM `formula1.constructor_unusual_drivers`
  WHERE race_year >= 1985
)

SELECT
  COUNTIF(num_drivers > 2) AS count_more_than_2,
  COUNTIF(num_drivers < 2) AS count_less_than_2,
  COUNT(*) AS total_unusual_cases,
  ROUND(COUNTIF(num_drivers > 2) * 100.0 / COUNT(*), 2) AS perc_more_than_2,
  ROUND(COUNTIF(num_drivers < 2) * 100.0 / COUNT(*), 2) AS perc_less_than_2
FROM post_1985_unusual_drivers

Query is running:   0%|          |

Downloading:   0%|          |

,count_more_than_2,count_less_than_2,total_unusual_cases,perc_more_than_2,perc_less_than_2
0,1,316,317,0.32,99.68


### 1.9 Grid stats and last queries before ML application in Databricks (3rd interesting insight)

In [43]:
%%bigquery
WITH winners AS (
  SELECT
    race_id,
    start_grid_position,
    wins
  FROM
    `formula1.joined_results`
  WHERE wins = 1
)

SELECT
  SUM(CASE WHEN start_grid_position BETWEEN 1 AND 6 THEN 1 ELSE 0 END) AS wins_from_top6,
  SUM(CASE WHEN start_grid_position NOT BETWEEN 1 AND 6 THEN 1 ELSE 0 END) AS wins_from_others,
  COUNT(*) AS total_wins,
  ROUND(SAFE_DIVIDE(SUM(CASE WHEN start_grid_position BETWEEN 1 AND 6 THEN 1 ELSE 0 END), COUNT(*)) * 100, 2) AS pct_wins_top6
FROM winners

Query is running:   0%|          |

Downloading:   0%|          |

,wins_from_top6,wins_from_others,total_wins,pct_wins_top6
0,1043,85,1128,92.46


In [44]:
%%bigquery
WITH win_data AS (
  SELECT
    start_grid_position,
    SUM(wins) AS wins
  FROM
    `formula1.joined_results`
  WHERE
    start_grid_position != 0
  GROUP BY
    start_grid_position
),
total AS (
  SELECT SUM(wins) AS total_wins FROM win_data
),
with_cumulative AS (
  SELECT
    w.start_grid_position,
    w.wins,
    SUM(w.wins) OVER (ORDER BY w.start_grid_position) AS cumulative_wins,
    t.total_wins
  FROM
    win_data w, total t
)

SELECT
  start_grid_position,
  wins,
  ROUND(100 * wins / total_wins, 2) AS win_rate,
  cumulative_wins,
  ROUND(100 * cumulative_wins / total_wins, 2) AS cumulative_win_rate
FROM
  with_cumulative
ORDER BY
  start_grid_position;

Query is running:   0%|          |

Downloading:   0%|          |

,start_grid_position,wins,win_rate,cumulative_wins,cumulative_win_rate
0,1,481,42.64,481,42.64
1,2,268,23.76,749,66.40
2,3,137,12.15,886,78.55
3,4,68,6.03,954,84.57
4,5,49,4.34,1003,88.92
5,6,40,3.55,1043,92.46
6,7,23,2.04,1066,94.50
7,8,17,1.51,1083,96.01
8,9,5,0.44,1088,96.45
9,10,12,1.06,1100,97.52


# 2. To visualize the details of the results click on the link below
[Open my Looker Studio Dashboard](https://lookerstudio.google.com/reporting/5d3818d5-e596-4fe2-bfea-6e9cb4a5e3ec)

# 3. Data Enrichment
[To see the Databricks side with the Machine Learning in-depth analysis click here](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/3706853530635392/2578662843473137/2314117943557060/latest.html)